# Plotting

`emsarray` can produce simple plots, to help you visualise your data quickly.
Plots can be made from any supported format.

In [ ]:
import emsarray

Set up the environment...

In [ ]:
# This makes the figures nice and big for this notebook
from matplotlib import pyplot as plt
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['savefig.dpi'] = 100

# The coastlines used in the maps have some bad polygons,
# which cause some warnings. Lets turn those off.
import warnings
warnings.filterwarnings(
    "ignore", message="Shapefile shape has invalid polygon",
    category=UserWarning, module="shapefile")

A quick demonstration before we get started!

In [ ]:
gbr4 = emsarray.tutorial.open_dataset('gbr4')
gbr4

The `botz` variable indicates the depth of the ocean floor at a particular cell.

The data will be plotted on a map, wih coastlines overlayed.
A colour ramp will be applied and automatically scaled to the range of the data.

In [ ]:
gbr4.ems.plot('botz', title='Depth of sea-bed')

## Selecting what to plot

A plot has to be made of a single layer.
For datasets with multiple depth layers and multiple time steps,
start by selecting just one slice from each of these extra dimensions.

The BRAN2020 example file contains multiple time steps and multiple depth layers.
We will need to select just one of each before we can make a plot.

In [ ]:
bran2020 = emsarray.tutorial.open_dataset('bran2020')
bran2020

Using `dataset.isel(...)` we can trim the data so we have just the surface layer at the first timestep.
After that we plot the `temp` variable:

In [ ]:
bran2020.isel(Time=0, st_ocean=0).ems.plot('temp')

Instead of selecting a subset of the data across the entire dataset,
you can slice a single data array and plot that variable directly.
To demonstrate, we can plot the last time step by selecting it on the data array:

In [ ]:
temp = bran2020['temp'].isel(Time=-1, st_ocean=0)
bran2020.ems.plot(temp)

## Plotting vectors

You can plot vector data using two variables that make up the *u* and *v* components of the vector.

In [ ]:
fraser = emsarray.tutorial.open_dataset('fraser')
fraser

In [ ]:
# Select the surface currents at the first timestep
fraser_0 = fraser.isel(time=0, k=-1)

fraser.ems.plot(
    vector=(fraser_0["u"], fraser_0["v"]),
    title="Surface current velocity")

This can be combined with a scalar plot:

In [ ]:
fraser.ems.plot(
    scalar=fraser_0["eta"],
    vector=(fraser_0["u"], fraser_0["v"]),
    title="Sea surface elevation and surface current velocty",)

## Plotting computed data

Any data can be plotted on the model grid, as long as the data has the same shape.
To demonstrate this, let us make a plot of surface temperature for our American friends.

In [ ]:
farhenheit_temp = fraser_0['temp'] * 1.8 + 32
farhenheit_temp.attrs.update({
    'long_name': 'Surface temperature (°F)',
    'units': 'degrees Farhenheit'
})
fraser.ems.plot(farhenheit_temp)

## Custom plots

`dataset.ems.plot()` is useful for making quick plots of your data,
but it does not create publication quality plots, and it does not expose any styling options.
The following example shows how make a custom plot with your own styling, base layers, and titles.
It uses a few tools provided by `emsarray` to extract the data and construct the polygons,
but otherwise does all of the styling manually.

In [ ]:
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import emsarray
import matplotlib.pyplot as plt
import numpy as np
from emsarray.plot import polygon_to_patch, bounds_to_extent
from emsarray.utils import datetime_from_np_time
from matplotlib.collections import PatchCollection
from shapely.geometry import box

# What to plot
dataset = emsarray.tutorial.open_dataset("fraser")
dataset = dataset.isel(time=0, k=-1)
scalar = 'eta'
vector = ('u', 'v')
title = "Sea surface height and surface current velocity"
datetime_stamp = datetime_from_np_time(dataset["time"].values).strftime("%Y-%m-%d %H:%M")

# Constrain the view to this region
view_area = box(152.3, -26.8, 154, -23.5)

# What coordinate reference system the data are in
data_crs = dataset.ems.data_crs

# What coordinate reference system to plot the data in.
# You can choose any projection that you want.
view_crs = ccrs.NearsidePerspective(
    central_longitude=view_area.centroid.x, central_latitude=view_area.centroid.y)

# Set up a figure and some axes, then set view bounds
figure = plt.figure(figsize=(8, 10), dpi=100)
axes = plt.subplot(projection=view_crs)
axes.set_title(title + "\n" + datetime_stamp)
axes.set_aspect(aspect='equal', adjustable='datalim')
axes.set_extent(bounds_to_extent(view_area.bounds), crs=data_crs)

# Add some grid lines
gridlines = axes.gridlines(draw_labels=True, color=(0.1, 0.1, 0.1, 0.1))
gridlines.top_labels = False
gridlines.right_labels = False

# Add a base image layer
axes.add_image(cimgt.Stamen(style='terrain-background', desired_tile_form='RGBA'), 9)
# axes.add_image(cimgt.Stamen(style='terrain-labels', desired_tile_form='RGBA'), 9)
figure.text(
    0.5, 0, "Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL.",
    va='bottom', ha='center', fontsize=8)

# Plot the data
patches = dataset.ems.make_patch_collection(scalar, edgecolor='face', cmap='plasma')
axes.add_collection(patches)
figure.colorbar(patches, ax=axes, location='right', label='metres')

quiver = dataset.ems.make_quiver(axes, *vector)
axes.add_collection(quiver)
plt.show()